In [1]:
import numpy as np
import pandas as pd
import os
import re
import tensorflow as tf
from tensorflow import keras
from pandas.plotting import table
import matplotlib.pyplot as plt
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam
tf.enable_eager_execution()
from DataLoader import Predictions_Input,map_CU,generate_Depth

/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tomas/Desktop/TCC/venv/lib/python3.6/site-packages/tensorf

In [2]:
Dados = Predictions_Input()
Input,Video = Dados.generate_input()

In [4]:
Depth_0 = keras.models.load_model('Model_Depth0.h5')
Depth_1 = keras.models.load_model('Model_Depth1.h5')
Depth_2 = keras.models.load_model('Model_Depth2.h5')

In [18]:
string_2 = ""
for frame in Input["Frame"].unique():
    Predict = Input.loc[Input["Frame"]==frame]
    file = open("./TapEncoder/Test_{}".format(frame),"w")
    for i in range(0,Predict.shape[0]):
        if ((Predict.iloc[i]["Height"]+64)<height):
            Input_Depth0 = map_CU(Predict.iloc[i],height,width,64,Video,QP)
            result = Depth_0.predict(Input_Depth0)
            string = str(result[0].argmax())
        else:
            string = "1"
        if string == "1" :
            fun = generate_Depth(Predict.iloc[i],32)
            for j in range(0,fun.shape[0]):
                if ((fun.iloc[j]["Height"]+32)<height):
                    Input_Depth1 = map_CU(fun.iloc[j],height,width,32,Video,QP)
                    result = Depth_1.predict(Input_Depth1)
                    check = str(result[0].argmax())
                    string = string + " " + check
                else:
                    string = string + " 1"
                    check = "1"
                if check == "1":
                    not_fun = generate_Depth(fun.iloc[j],16)
                    for k in range(0,not_fun.shape[0]):
                        if ((not_fun.iloc[k]["Height"]+16)<height):
                            Input_Depth2 = map_CU(not_fun.iloc[k],height,width,16,Video,QP)
                            result = Depth_2.predict(Input_Depth2)
                            string_2 = string_2 + " " + str(result[0].argmax())
                        else:
                            string_2 = string_2 + " 1"
        string = string + string_2
        string_2 = ""
        file.write("CU ({:4d},{:4d}) - {}\n".format(Predict.iloc[i]["Height"],Predict.iloc[i]["Width"],string))